In [52]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv


In [53]:
train = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv", sep=',')

In [54]:
test = pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")

In [55]:
train['text']

0       Our Deeds are the Reason of this #earthquake M...
1                  Forest fire near La Ronge Sask. Canada
2       All residents asked to 'shelter in place' are ...
3       13,000 people receive #wildfires evacuation or...
4       Just got sent this photo from Ruby #Alaska as ...
                              ...                        
7608    Two giant cranes holding a bridge collapse int...
7609    @aria_ahrary @TheTawniest The out of control w...
7610    M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...
7611    Police investigating after an e-bike collided ...
7612    The Latest: More Homes Razed by Northern Calif...
Name: text, Length: 7613, dtype: object

In [56]:
train['target'].value_counts()

target
0    4342
1    3271
Name: count, dtype: int64

In [57]:
from transformers import pipeline

model_path = "cardiffnlp/twitter-roberta-base-sentiment-latest"

pipe = pipeline(
    model = model_path,
    tokenizer = model_path,
    return_all_scores=True,
    device="cuda:0"
)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0
/usr/local/lib/python3.11/dist-packages/transformers/pipelines/text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return

In [58]:
train['target']

0       1
1       1
2       1
3       1
4       1
       ..
7608    1
7609    1
7610    1
7611    1
7612    1
Name: target, Length: 7613, dtype: int64

In [59]:
import numpy as np
from tqdm import tqdm
from transformers.pipelines.pt_utils import KeyDataset

# 추론
texts =train["text"].tolist()
y_pred = []

for output in tqdm(pipe(texts, batch_size=32), total=len(texts)):
    negative_score = output[0]["score"]  # output은 dict 하나임. (pipeline이 iterable이면 리스트 반환, list 입력이면 dict)
    positive_score = output[2]["score"]
    assignment = np.argmax([negative_score,positive_score])
    y_pred.append(assignment)

100%|██████████| 7613/7613 [00:00<00:00, 227789.02it/s]


In [60]:
from sklearn.metrics import classification_report

def evaluate_performance(y_true, y_pred):
    performance = classification_report(
        y_true, y_pred,
        target_names = ["Negative Review", "Positive Review"]
    )
    print(performance)

In [61]:
evaluate_performance(train["target"], y_pred)

                 precision    recall  f1-score   support

Negative Review       0.48      0.58      0.52      4342
Positive Review       0.23      0.16      0.19      3271

       accuracy                           0.40      7613
      macro avg       0.35      0.37      0.36      7613
   weighted avg       0.37      0.40      0.38      7613



In [62]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")

train_embeddings = model.encode(train['text'], show_progress_bar=True)
test_embeddings = model.encode(test['text'], show_progress_bar=True)

Batches:   0%|          | 0/238 [00:00<?, ?it/s]

Batches:   0%|          | 0/102 [00:00<?, ?it/s]

In [63]:
train_embeddings.shape

(7613, 768)

In [64]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(random_state=42)
clf.fit(train_embeddings, train['target'])

LogisticRegression(random_state=42)

In [65]:
'''
y_pred = clf.predict(test_embeddings)
# 제출 파일 생성
submission = test[["id"]].copy()
submission["target"] = y_pred

# CSV 저장
submission.to_csv("submission.csv", index=False)
'''

'\ny_pred = clf.predict(test_embeddings)\n# 제출 파일 생성\nsubmission = test[["id"]].copy()\nsubmission["target"] = y_pred\n\n# CSV 저장\nsubmission.to_csv("submission.csv", index=False)\n'

In [66]:
from sklearn.metrics.pairwise import cosine_similarity

df = pd.DataFrame(np.hstack([train_embeddings,np.array(train['target']).reshape(-1,1)]))
averaged_target_embeddings = df.groupby(768).mean().values

sim_matrix = cosine_similarity(test_embeddings, averaged_target_embeddings)
y_pred = np.argmax(sim_matrix, axis=1)

In [67]:
'''
# 제출 파일 생성
submission = test[["id"]].copy()
submission["target"] = y_pred

# CSV 저장
submission.to_csv("submission.csv", index=False)
'''

'\n# 제출 파일 생성\nsubmission = test[["id"]].copy()\nsubmission["target"] = y_pred\n\n# CSV 저장\nsubmission.to_csv("submission.csv", index=False)\n'

In [68]:
pipe = pipeline(
    "text2text-generation",
    model = "google/flan-t5-small",
    device = "cuda:0"
)

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


In [69]:
prompt = "Is the following sentence positive or nagative?"
test["t5"] = prompt + test["text"]
test

,id,keyword,location,text,t5
0,0,NaN,NaN,Just happened a terrible car crash,Is the following sentence positive or nagative...
1,2,NaN,NaN,"Heard about #earthquake is different cities, s...",Is the following sentence positive or nagative...
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are...",Is the following sentence positive or nagative...
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires,Is the following sentence positive or nagative...
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan,Is the following sentence positive or nagative...
...,...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...,Is the following sentence positive or nagative...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...,Is the following sentence positive or nagative...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...,Is the following sentence positive or nagative...
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...,Is the following sentence positive or nagative...


In [70]:
y_pred = []
for output in tqdm(pipe(test["t5"].tolist()), total=len(test)):
    text = output["generated_text"]
    y_pred.append(0 if text == "negative" else 1)

KeyboardInterrupt: 

In [ ]:
submission = test[["id"]].copy()
submission["target"] = y_pred

# CSV 저장
submission.to_csv("submission.csv", index=False)